## Linear Activity vs Dyskinesia and multi-freq Biomarker Prediction


In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import csv
from dataclasses import dataclass, field, fields
from itertools import compress, product
import pandas as pd
import numpy as np
import sklearn as sk
from scipy import signal, stats

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [ ]:
projectpath = get_project_path_in_notebook()


Import own functions

In [ ]:
os.chdir(os.path.join(projectpath, 'code'))

# own utility functions
import utils.utils_fileManagement as utilsFiles

# own data exploration functions
import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_analysis.ft_processing_helpers as ftProc
import lfpecog_analysis.psd_lid_stats as lidStats
import lfpecog_analysis.ft_processing_helpers as ftProc
import lfpecog_features.feats_spectral_helpers as specHelp
import lfpecog_analysis.get_acc_task_derivs as getAccTask

import lfpecog_predict.prepare_predict_arrays as predArrays
import lfpecog_features.extract_ssd_features as ssdFeats
import lfpecog_analysis.stats_fts_lid_corrs as ft_stats
from lfpecog_plotting.plotHelpers import get_colors
import lfpecog_plotting.plotHelpers as pltHelp
import lfpecog_plotting.plot_SSD_feat_descriptives as plot_ssd_descr

check versions

In [ ]:
from  matplotlib import __version__ as plt_version

# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', plt_version)
# Python sys 3.9.0 (default, Nov 15 2020, 08:30:55) [MSC v.1916 64 bit (AMD64)]
# pandas 1.4.4
# numpy 1.23.3
# matplotlib 3.5.3

### Import data

imports SSD-band-envelops with parallel CDRS, timestamps, task, movement-coding

In [ ]:
# SET VERSIONS

FT_VERSION='v8'
SETTINGS = utilsFiles.load_ft_ext_cfg(FT_VERSION=FT_VERSION)

SUBS = utilsFiles.get_avail_ssd_subs(
    DATA_VERSION=SETTINGS["DATA_VERSION"],
    FT_VERSION=FT_VERSION,
)
print(f'n = {len(SUBS)} subjects available')

## FIG: Spectral Scatterplot: Movement vs. Dyskinesia dependence

Load STN data (n=21)

In [ ]:
FT_VERSION = 'v8'
INCL_CORE_CDRS = True
CATEG_CDRS = False
INCL_ECOG = False  # if True, STN-only patients are NOT included
MILD_CDRS = 4  # cut off for dyskinesia categorization (4)
SEV_CDRS = 8  # cut off for dyskinesia categorization (8)

# ### Create Feature Class
# FeatLid_STN = ftProc.FeatLidClass(
#     FT_VERSION=FT_VERSION,
#     CDRS_RATER='Patricia',
#     INCL_ECOG=INCL_ECOG,
#     INCL_ACC_RMS=False,
#     CATEGORICAL_CDRS=CATEG_CDRS,
#     CORR_TARGET='CDRS',
#     cutMild=MILD_CDRS, cutSevere=SEV_CDRS,
#     TO_CALC_CORR=False,
# )

### LOAD existing classes with features and labels
featLabPath = os.path.join(utilsFiles.get_project_path('data'),
                           'prediction_data',
                           'featLabelClasses')
stn_pickle = f'featLabels_ft{FT_VERSION}_Cdrs_StnOnly.P'

FeatLid_STN = utilsFiles.load_class_pickle(
    os.path.join(featLabPath, stn_pickle),
    convert_float_np64=True
)


# MERGE GAMMA1-2-3 FEATURES
for sub in FeatLid_STN.FEATS:
    FeatLid_STN.FEATS[sub] = ft_stats.replace_gammas_for_meanGamma(FeatLid_STN.FEATS[sub])



In [ ]:
### Load Feature Class for Acc (v6)

### Create 
# Acc_STN = ftProc.FeatLidClass(
#     FT_VERSION='v6',
#     CDRS_RATER='Patricia',
#     INCL_ECOG=INCL_ECOG,
#     INCL_ACC_RMS=True,
#     CATEGORICAL_CDRS=CATEG_CDRS,
#     CORR_TARGET='CDRS',
#     cutMild=MILD_CDRS, cutSevere=SEV_CDRS,
#     TO_CALC_CORR=False,
# )


# Load v6 to get ACC-values
Acc_STN = utilsFiles.load_class_pickle(
    os.path.join(featLabPath, stn_pickle.replace('v8', 'v6')),
    convert_float_np64=True
)

Load ECoG data (n=13)

In [ ]:
FT_VERSION = 'v8'
INCL_CORE_CDRS = True
CATEG_CDRS = False
INCL_ECOG = True  # if True, STN-only patients are NOT included
MILD_CDRS = 4  # cut off for dyskinesia categorization
SEV_CDRS = 8  # cut off for dyskinesia categorization

### Create Feature Class
# FeatLid_ECOG = ftProc.FeatLidClass(
#     FT_VERSION=FT_VERSION,
#     CDRS_RATER='Patricia',
#     INCL_ECOG=INCL_ECOG,
#     INCL_ACC_RMS=False,
#     CATEGORICAL_CDRS=CATEG_CDRS,
#     CORR_TARGET='CDRS',
#     cutMild=MILD_CDRS, cutSevere=SEV_CDRS,
#     TO_CALC_CORR=False,
# )

### LOAD existing classes with features and labels
featLabPath = os.path.join(utilsFiles.get_project_path('data'),
                           'prediction_data',
                           'featLabelClasses')
ecog_pickle = f'featLabels_ft{FT_VERSION}_Cdrs_Ecog.P'

FeatLid_ECOG = utilsFiles.load_class_pickle(
    os.path.join(featLabPath, ecog_pickle),
    convert_float_np64=True
)
# MERGE GAMMA1-2-3 FEATURES
for sub in FeatLid_ECOG.FEATS:
    FeatLid_ECOG.FEATS[sub] = ft_stats.replace_gammas_for_meanGamma(FeatLid_ECOG.FEATS[sub])



# Load v6 to get ACC-values
Acc_ECOG = utilsFiles.load_class_pickle(
    os.path.join(featLabPath, ecog_pickle.replace('v8', 'v6')),
    convert_float_np64=True
)


Get task data per minute

In [ ]:
TASK_MINS = getAccTask.get_task_minutes(LOAD_JSON=True, SUBS=SUBS)

In [ ]:
import lfpecog_plotting.plot_Spectrals_vs_LID as plotSpecLid

Plot Scatterplot

In [ ]:
importlib.reload(ftProc)
importlib.reload(plotSpecLid)

FIG_DATE = '1009'

POW_or_COH = 'COH'  # POW or COH
SRC = 'ecog'  # ecog or lfp
EXCL_FREE = False
gamma_peak = 'peak'  # peak / mean

if SRC == 'lfp':
    FeatClass = FeatLid_STN  # FeatLid_ECOG
    AccClass = Acc_STN
if SRC == 'ecog':
    FeatClass = FeatLid_ECOG
    AccClass = Acc_ECOG

FIG_NAME = (f'{FIG_DATE}_PowerScatter_{SRC.upper()}_LID_MOVE_g{gamma_peak}'
            f'_n{len(FeatClass.FEATS.keys())}_ft{FT_VERSION}_shf')
if EXCL_FREE: FIG_NAME = 'woFREE_' + FIG_NAME

plotSpecLid.scatter_Feats_LID_MOVE(
    FeatClass=FeatClass,
    AccClass=AccClass,
    POW_or_COH=POW_or_COH,
    SRC=SRC,
    SHUFFLE_SCATTERS=True,
    SAVE_FIG=True,
    FIG_NAME=FIG_NAME,
    gamma_mean_or_peakband=gamma_peak,
    EXCL_FREE=EXCL_FREE,
    task_minutes=TASK_MINS,
    ZERO_SPACE=True,
)

## FIG) Simple biomarker versus Dyskinesia onset


Calculates based on envelop arrays from predArrays.get_move_selected_env_arrays()

In [ ]:
from lfpecog_analysis.psd_analysis_classes import PSD_vs_Move_sub

In [ ]:
importlib.reload(predArrays)

# get move-selected env arrays
# contains: freq-bands, CDRS, timestamps, tasks, mov-coding

# TODO: FIRST extract via cmd: lfpecog_predict.prepare_precit_arrays
DATA, env_fbands = {}, {}

for sub in SUBS:
    DATA[sub], env_fbands[sub] = predArrays.get_move_selected_env_arrays(
        sub=sub, LOAD_SAVE=True,
        FT_VERSION='v8',
    )

Prepare data

In [ ]:
src = 'lfp_left'

ex_sub = list(env_fbands.keys())[0]
ex_src= list(env_fbands[ex_sub].keys())[0]

i_theta = np.where([k == 'theta' for k in env_fbands[sub][src]])[0][0]
i_beta = np.where([k == 'lo_beta' for k in env_fbands[sub][src]])[0][0]
i_gammaPeak = np.where([k == 'gammaPeak' for k in env_fbands[sub][src]])[0][0]
i_cdrs = len(env_fbands[sub][src])
i_time = i_cdrs + 1
i_task = i_time + 1
# i_move is last

time_list = {'lid': [], 'nolid': []}
ratio_list = {'lid': [], 'nolid': []}

lid_onsets = []

sub_list = []

for sub, src in product(DATA.keys(),
                        ['lfp_left', 'lfp_right']):
    
    if sum(DATA[sub][src][i_cdrs, :]) == 0:
        print(f'sub {sub} had no LID')
        subgroup = 'nolid'
    else:
        subgroup = 'lid'

    print(f'...calc {sub, src}')
    
    idx_sort = np.argsort(DATA[sub][src][i_time, :])
    sort_arr = DATA[sub][src][:, idx_sort]

    theta = sort_arr[i_theta, :]
    beta = sort_arr[i_beta, :]
    gamma = sort_arr[i_gammaPeak, :]


    for arr in [theta, beta, gamma]:
        off_sel = sort_arr[i_time, :] < 5
        m = np.mean(arr[off_sel])
        sd = np.std(arr[off_sel])
        arr = (arr - m) / sd

    assert theta.shape == beta.shape, 'shapes unequal'

    # sets all times to ZERO at LID onset
    if subgroup == 'lid':
        i0_lid = np.where(sort_arr[i_cdrs, :] > 0)[0][0]
        t0_lid = sort_arr[i_time, i0_lid]
        lid_times = sort_arr[i_time, :] - t0_lid
        lid_onsets.append(t0_lid)
    else:
        lid_times = sort_arr[i_time, :]


    time_list[subgroup].append(lid_times)
    ratio = (theta / beta) * gamma
    ratio = (ratio - np.mean(ratio)) / np.std(ratio)
    ratio_list[subgroup].append(ratio)

    sub_list.append(f'{sub}_{src}')


In [ ]:
min_sec, max_sec = (
    int(np.min([np.min(l) for l in time_list['lid']])),
    int(np.max([np.max(l) for l in time_list['lid']]))
)

print(f'mean LID onset: {round(np.mean(lid_onsets) / 60, 1)} minutes'
      f' (sd: {round(np.std(lid_onsets) / 60, 1)})')
# correct group without LID to comparable time offsets
nolid_new_times = []
for t in time_list['nolid']:
    nolid_new_times.append(t - np.mean(lid_onsets))


In [ ]:
# create overall array per X seconds
WIN_LEN = 10

## LID group
min_sec, max_sec = (
    int(np.min([np.min(l) for l in time_list['lid']])),
    int(np.max([np.max(l) for l in time_list['lid']]))
)
t_new = np.arange(min_sec, max_sec, WIN_LEN)

ratio_arr = np.array([[np.nan] * len(t_new)] * len(ratio_list['lid']))

for i_t, t0 in enumerate(t_new):

    for i_row, (sig_temp, t_temp) in enumerate(
        zip(ratio_list['lid'], time_list['lid'])
    ):
        # select idx for window
        win_sel = np.logical_and(t_temp > t0, t_temp < (t0 + WIN_LEN))
        # add mean ratio to correct idx
        ratio_arr[i_row, i_t] = np.mean(sig_temp[win_sel])


## NO-LID group

# correct group without LID to comparable time offsets
nolid_new_times = []
for t in time_list['nolid']:
    nolid_new_times.append(t - np.mean(lid_onsets))

ratio_arr_noLID = np.array([[np.nan] * len(t_new)] * len(ratio_list['nolid']))

# use same time frame
for i_t, t0 in enumerate(t_new):

    for i_row, (sig_temp, t_temp) in enumerate(
        zip(ratio_list['nolid'], nolid_new_times)
    ):
        # select idx for window
        win_sel = np.logical_and(t_temp > t0, t_temp < (t0 + WIN_LEN))
        # add mean ratio to correct idx
        ratio_arr_noLID[i_row, i_t] = np.mean(sig_temp[win_sel])
        

Plot Ratio over Time vs LID-onset

In [ ]:
importlib.reload(plotSpecLid)

# plot_ratio_biomarker(ratio_arr=ratio_arr,
#                      Z_SCORE_RATIOS=False,
#                      MIN_SUBS=5,
#                      SMOOTH_WIN=0,)

plotSpecLid.plot_ratio_biomarker(
    ratio_arr=ratio_arr,
    t_new=t_new,
    Z_SCORE_RATIOS=False,
    MIN_SUBS=5,
    SMOOTH_WIN=10,
    SAVE_FIG=False,
)

# plot_ratio_biomarker(ratio_arr=ratio_arr_noLID,
#                      Z_SCORE_RATIOS=False, MIN_SUBS=2)

Calculate binary AUROC

In [ ]:
# remove no-lid-subjects from sub_list
sub_list = [s for s in sub_list if 
            all(c not in s for c in ['101', '017', '109'])]

thresholds = np.arange(-.5, .9, .001)

auroc = []

for sub in np.unique([s.split('_')[0] for s in sub_list]):
    
    sub_roc = []
    
    # print(f'sub-{sub}')
    sub_sel = [sub in s for s in sub_list]
    sub_arr = np.mean(ratio_arr.copy()[sub_sel], axis=0)
    sub_t = t_new.copy()
    sel = [not np.isnan(v) for v in sub_arr]
    sub_arr = sub_arr[sel]
    sub_t = sub_t[sel]
    sub_y_true = (sub_t > 0).astype(int)

    for t in thresholds:
        y_pred = (sub_arr > t).astype(int)
        tp = sum(np.logical_and(y_pred == 1, sub_y_true == 1))
        fn = sum(np.logical_and(y_pred == 0, sub_y_true == 1))
        tpr = tp / (tp + fn)
        fp = sum(np.logical_and(y_pred == 1, sub_y_true == 0))
        tn = sum(np.logical_and(y_pred == 0, sub_y_true == 0))
        fpr = fp / (fp + tn)
        sub_roc.append([fpr, tpr])
    
    auroc.append(sub_roc)

    

In [ ]:
# Plot single AUROCs
for roc in auroc:
    roc = np.array(roc)
    plt.plot(roc[:, 0], roc[:, 1], alpha=.3,)

plt.xlim(-.05, 1.05)
plt.ylim(-.05, 1.05)

# change level
plt.plot([0, 1], [0, 1], lw=3, alpha=.8, color='gray',
         ls='--',)

plt.xlabel('False pos. rate', size=16, weight='bold',)
plt.ylabel('True pos. rate', size=16, weight='bold',)

plt.title('AUROC based on [ \u03B8 * \u03B2 / \u03B3 ] ratio',
          size=16, weight='bold',)

FIG_PATH = os.path.join(utilsFiles.get_project_path('figures'),
                            'final_Q1_2024',
                            'prediction', 'ratio')
            
plt.savefig(os.path.join(FIG_PATH, '000_RATIO_AUROC_v6'),
            dpi=300, facecolor='w',)

plt.close()

In [ ]:
# Plot Mean AUROC
SAVEFIG=True
FIG_DATE='0918'

all_aurocs = np.array(auroc)

mean_roc = np.mean(all_aurocs, axis=0)
sd_roc = np.std(all_aurocs, axis=0)

plt.plot(mean_roc[:, 0], mean_roc[:, 1],
         c='k', alpha=.8, lw=3,)

plt.fill_between(x=mean_roc[:, 0],
                 y1=mean_roc[:, 1] - sd_roc[:, 1],
                 y2=mean_roc[:, 1] + sd_roc[:, 1],
                 color='gray', alpha=.4, lw=0,)

plt.xlim(-.05, 1.05)
plt.ylim(-.05, 1.05)

# change level
plt.plot([0, 1], [0, 1], lw=3, alpha=.8, color='gray',
         ls='--',)

plt.xlabel('False pos. rate', size=16, weight='bold',)
plt.ylabel('True pos. rate', size=16, weight='bold',)

plt.title('AUROC based on [ \u03B8 * \u03B2 / \u03B3 ] ratio',
          size=16, weight='bold',)

FIG_PATH = os.path.join(utilsFiles.get_project_path('figures'),
                            'final_Q1_2024',
                            'prediction', 'ratio')
if SAVEFIG:          
    plt.savefig(os.path.join(FIG_PATH, f'{FIG_DATE}_RATIO_meanAUROC_v6'),
                dpi=300, facecolor='w',)
    plt.close()
else:
    plt.show()

### Leave-one-subject-out Cross-Validation of Ratio-Biomarker

Prepare Data: mean LFP Ratio with binary-LID for every subject

- single spectral bands are standardised against their first 5 minutes
- ecpohed mean-envelop values are saved as json files per STN (L vs R)
- epoch length is variable (default 10 seconds)
- envelop-ratio calculated on high-sampling rate, then per hemisphere ecpohed mean, then mean over both hemispheres for identical epoch-times (rounded on 10 seconds)

In [ ]:
import json


In [ ]:
def load_ratios_json(sub, src, version='v1',):

    try:
        path = os.path.join(projectpath, 'data', 'prediction_data', 'ratio_marker')
        if version == 'v2':
            path = os.path.join(projectpath, 'data', 'prediction_data', 'ratio_marker_v2')
        f_name = f'{sub}_{src}_ratio_data.json'

        with open(os.path.join(path, f_name), 'r') as f:
            temp = json.load(f)
        
        print(f'successfully loaded loading {f_name} from {path}!')

        return True, temp

    except:
        print(f'could not load RATIO-json {f_name} from {path}')

        return False, False


def save_ratios_json(sub, src, data_dict, version='v1'):

    path = os.path.join(projectpath, 'data', 'prediction_data', 'ratio_marker')
    if version == 'v2':
        path = os.path.join(projectpath, 'data', 'prediction_data', 'ratio_marker_v2')
    f_name = f'{sub}_{src}_ratio_data.json'

    with open(os.path.join(path, f_name), 'wt') as f:
        json.dump(data_dict, f)

In [ ]:
"""
consider v2 with unilateral z-scoring (line )
"""
UNILAT_STD = False
RATIO_V = 'v1'

WIN_LEN = 10

ratio_arrays = {}  # collect mean ratios left-right STN
lid_arrays = {}  # collect mean ratios left-right STN

# for sub in DATA.keys():
#     temp_ratio_arrays[sub] = {}
#     temp_time_arrays[sub] = {}

lid_onsets = []

sub_list = []

for sub in DATA.keys():

    temp_ratio = {}  # collect mean ratios left-right STN
    temp_times = {}  # collect timestamps to check left-right match
    temp_lid = {}

    for src in ['lfp_left', 'lfp_right']:

        LOAD_SUCCESS, temp_dict = load_ratios_json(sub=sub, src=src, version=RATIO_V,)

        if LOAD_SUCCESS:
            assert temp_dict["sub"] == sub, f'incorrect sub loaded {temp_dict[sub]} vs {sub}'
            assert temp_dict["src"] == src, f'incorrect sub loaded {temp_dict[src]} vs {src}'
            temp_ratio[src] = np.array(temp_dict['ratio'])
            temp_times[src] = np.array(temp_dict['times'])
            temp_lid[src] = np.array(temp_dict['lid'])

            continue
        
        # IF LOADING WAS NOT successful....:

        print(f'...calc {sub, src}')

        i_theta = np.where([k == 'theta' for k in env_fbands[sub][src]])[0][0]
        i_beta = np.where([k == 'lo_beta' for k in env_fbands[sub][src]])[0][0]
        i_gammaPeak = np.where([k == 'gammaPeak' for k in env_fbands[sub][src]])[0][0]
        i_cdrs = len(env_fbands[sub][src])
        i_time = i_cdrs + 1
        i_task = i_time + 1

        idx_sort = np.argsort(DATA[sub][src][i_time, :])
        sort_arr = DATA[sub][src][:, idx_sort]

        # get spectral bands
        theta = sort_arr[i_theta, :]
        beta = sort_arr[i_beta, :]
        gamma = sort_arr[i_gammaPeak, :]

        # standardise bands separately (against 'OFF')
        for arr in [theta, beta, gamma]:
            off_sel = sort_arr[i_time, :] < 5
            m = np.mean(arr[off_sel])
            sd = np.std(arr[off_sel])
            arr = (arr - m) / sd

        assert theta.shape == beta.shape, 'shapes unequal'

        # add unilateral ratio, not z-scored
        ratio = (theta / beta) * gamma
        if UNILAT_STD: ratio = (ratio - np.mean(ratio)) / np.std(ratio)

        # define binary LID
        temp_lid[src] = sort_arr[i_cdrs, :] > 0
        
        # add times to check matching sides
        temp_times[src] = sort_arr[i_time, :]
    
        # MERGE VALUES INTO X-sec EPOCHS
        print(f'...START EPOCHING {src} in {WIN_LEN}-sec epochs')
        time_range = np.arange(round(min(temp_times[src]), -1),
                          round(max(temp_times[src]), -1), WIN_LEN)
        new_ratios, new_lid, new_times = [], [], []
        
        print('min-max-time', np.min(temp_times[src]), np.max(temp_times[src]))
        t_start = round(min(temp_times[src]), -1)

        while (t_start + WIN_LEN) < round(max(temp_times[src]), -1):
            # print('new loop:', t_start)
            t_start = round(t_start, -1)
            t_end = t_start + WIN_LEN
            i_start = np.argmin(abs(temp_times[src] - t_start))
            i_end = np.argmin(abs(temp_times[src] - t_end))
            
            if (i_end - i_start) < 2000:
                t_start = temp_times[src][i_end + 2000]
                print(f'too small epoch, jump to time = {t_start}')
                continue
            
            print('adding..........', t_start, t_end, (i_end-i_start))
            new_ratios.append(np.mean(ratio[i_start:i_end]))
            new_lid.append(any(temp_lid[src][i_start:i_end] > 0))
            new_times.append(t_start)
            t_start += WIN_LEN


        # STORE PER SUB SOURCE IN PICKLE
        ind_src_dict = {
            "sub": sub,
            "src": src,
            "times": new_times,
            "lid": new_lid,
            "ratio": new_ratios
        }
        save_ratios_json(sub=sub, src=src, data_dict=ind_src_dict, version=RATIO_V,)

        # replace with epoched values
        temp_ratio[src] = np.array(new_ratios)
        temp_lid[src] = np.array(new_lid)
        temp_times[src] = np.array(new_times)

    ### Get MEAN ratios L/R (after left and right arrays created)
    sub_lid = temp_lid['lfp_left']
    sub_times = temp_times['lfp_left']  # for posthoc LID-binary correction based on time

    if len(temp_ratio['lfp_left']) != len(temp_ratio['lfp_right']):
        print(f'correct different ratio lengths L/R')
        # INCLUDE ONLY MUTUAL TIMES!
        incl_left = np.isin(temp_times['lfp_left'],
                            temp_times['lfp_right'])
        incl_right = np.isin(temp_times['lfp_right'],
                             temp_times['lfp_left'])
        
        # select ratio values corresponding to mutual times
        temp_ratio['lfp_left'] = temp_ratio['lfp_left'][incl_left]
        temp_ratio['lfp_right'] = temp_ratio['lfp_right'][incl_right]
        
        # select binary lid corr to times and ratios
        sub_lid = temp_lid['lfp_left'][incl_left]
        sub_times = temp_times['lfp_left'][incl_left]
        
    sub_mratio = np.mean([temp_ratio['lfp_left'],
                          temp_ratio['lfp_right']], axis=0)
    
    assert len(sub_mratio) == len(sub_lid)

    sub_mratio = (sub_mratio - np.mean(sub_mratio)) / np.std(sub_mratio)  # z-score mean ratio
    ratio_arrays[sub] = sub_mratio  # add zscored mean ratio per sub

    if RATIO_V == 'v1':  # correct false binary values
        cdrs_times, cdrs_scores = importClin.get_cdrs_specific(sub=sub, INCL_CORE_CDRS=True,)
        cdrs_times *= 60  # correct minutes to seconds
        corr_lid = [cdrs_scores[np.argmin(abs(cdrs_times - t))] for t in sub_times]  # match ratio times to closest-cdrs
        sub_lid = np.array(corr_lid) > 0

    lid_arrays[sub] = sub_lid  # add binary dysk per sub

In [ ]:
corr_lid = [cdrs_scores[np.argmin(abs(cdrs_times - t))] for t in sub_times]
# for t in sub_times:
#     cdrs_i = np.argmin(abs(cdrs_times - t))
#     print(cdrs_scores[cdrs_i])

plt.plot(sub_times, np.array(corr_lid) > 0)

In [ ]:
cdrs_times, cdrs_scores = importClin.get_cdrs_specific(sub=sub, INCL_CORE_CDRS=True,)
cdrs_times *= 60
plt.plot(cdrs_times, cdrs_scores)
lid_i0 = np.where(cdrs_scores > 0)[0][0]
# correct for seconds
# lid_s0 = 
print(cdrs_times[lid_i0])

Perform LOSO Cross Validation

In [ ]:
from sklearn.metrics import balanced_accuracy_score

In [ ]:
cv_opt_thresh = []
cv_test_acc = []

for test_sub in ratio_arrays.keys():
    print(f'\n.....TEST sub-{sub}')
    # get values for test sub
    test_X = ratio_arrays[test_sub]
    test_y = lid_arrays[test_sub]

    # get all values for other subs as training
    train_X, train_y_true = [], []

    for train_sub in ratio_arrays.keys():
        if train_sub == test_sub:
            print(f'... no including sub-{train_sub}')
            continue
        print(f'include sub-{train_sub} for training :)')

        train_X.extend(ratio_arrays[train_sub])
        train_y_true.extend(lid_arrays[train_sub])
    
    # Training: finding optimal threshold
    train_perf = []
    threshes = np.arange(-2, 2, .01)

    for thresh in threshes:
        y_train_pred = train_X > thresh
        # get balanced accuracy
        thresh_acc = balanced_accuracy_score(y_train_true,
                                             y_train_pred)
        train_perf.append(thresh_acc)
        print(f'...train (thresh: {thresh}) acc: {thresh_acc}')
    
    # find optimal threshold
    i_opt_thresh = np.argmax(train_perf)
    cv_opt_thresh.append(threshes[i_opt_thresh])
    cv_test_acc.append(train_perf[i_opt_thresh])

    print(f'\ntest-performance sub-{test_sub}: {train_perf[i_opt_thresh]}'
          f' (threshold: {threshes[i_opt_thresh]})\n')

In [ ]:
len()

## Old)

Extract Spectral Power and Variation in Feature windows

TODO:
- current gamma: only indiv peak, add sum gamma over 60 - 90

In [ ]:
def indiv_zscoring_feats(X_arr, sub_arr):

    for i_f, sub in product(np.arange(X_arr.shape[1]),
                            np.unique(sub_arr)):
        # loop over all feature and sub combinations
        sub_sel = sub_arr == sub
        m = np.mean(X_arr[sub_sel, i_f])
        sd = np.std(X_arr[sub_sel, i_f])
        # z-score values for sub
        X_arr[sub_sel, i_f] = (X_arr[sub_sel, i_f] - m) / sd
    
    return X_arr

In [ ]:
importlib.reload(ssdFeats)

X_arrs, y_arrs, sub_arrs = {}, {}, {}
mov_dep_code = {}

for i_mov, MOV_SEL in enumerate(['INDEP', 'DEPEND']):
    # loading/ creating/ saving in ssdFeats script
    (
        X_arrs[i_mov], y_arrs[i_mov], sub_arrs[i_mov], feat_names
    ) = ssdFeats.get_moveSpec_predArrays(
        MOV_SEL=MOV_SEL, LOAD_SOURCES=['STN',],
        POWER_METHOD='ENV'
    )
    # add movement code
    mov_dep_code[i_mov] = np.array([i_mov] * X_arrs[i_mov]['STN'].shape[0]).T

# merging for zscoring together
stn_X = np.concatenate([X_arrs[0]['STN'], X_arrs[1]['STN']], axis=0)
stn_y = np.concatenate([y_arrs[0]['STN'], y_arrs[1]['STN']], axis=0)
stn_subids = np.concatenate([sub_arrs[0]['STN'], sub_arrs[1]['STN']], axis=0)

mov_dep_code = np.concatenate([mov_dep_code[0], mov_dep_code[1]], axis=0)

In [ ]:
# calculate coefficients and pvalues
CDRS_categs = {0: 'none', 1: 'mild',
               2: 'moderate', 3: 'severe'}

X = stn_X.copy()
y = stn_y.copy()
sub_ids = stn_subids.copy()

stat_arr = {'INDEP': {'coef': [], 'pval': []},
            'DEPEND': {'coef': [], 'pval': []}}

X = indiv_zscoring_feats(X, sub_arr=sub_ids)

for i_ft, ft in enumerate(feat_names['STN']):
    

    for i_mov, MOV_SEL in enumerate(['INDEP', 'DEPEND']):
        mov_bool = mov_dep_code == i_mov
        # define X and groups for feat
        ft_temp = X[mov_bool, i_ft]
        y_temp = y[mov_bool]
        box_categs = [ft_temp[y_temp == cat]
                      for cat in CDRS_categs.keys()]
        # run LMM
        coeff, pval = lidStats.run_mixEff_wGroups(
            dep_var=ft_temp,
            indep_var=y_temp,
            groups=sub_ids[mov_bool],
            TO_ZSCORE=False,
        )
        # save in dict-lists
        stat_arr[MOV_SEL]['coef'].append(coeff)
        stat_arr[MOV_SEL]['pval'].append(pval)

        

In [ ]:
# plot boxplots per feature

FIG_NAME = '0129env_boxplots_specPowerVar_vs_LIDcategs'


fig, axes = plt.subplots(len(feat_names['STN']), 2,
                         figsize=(12, 18),
                         sharex='col', sharey='row')

for i_ft, ft in enumerate(feat_names['STN']):
    
    for i_mov, MOV_SEL in enumerate(['INDEP', 'DEPEND']):
        # get boxplot data
        mov_bool = mov_dep_code == i_mov
        # define X and groups for feat
        ft_temp = X[mov_bool, i_ft]
        y_temp = y[mov_bool]
        box_categs = [ft_temp[y_temp == cat]
                      for cat in CDRS_categs.keys()]
        
        # stats are calculated before

        # plotting
        coeff = stat_arr[MOV_SEL]['coef'][i_ft]
        pval = stat_arr[MOV_SEL]['pval'][i_ft]
        axes[i_ft, i_mov].boxplot(box_categs)
        if pval < (.05 / len(feat_names['STN'])): w = 'bold'
        else: w='normal'
        axes[i_ft, i_mov].set_title(f'{ft}, mov-{MOV_SEL}\n'
                f'(coeff {round(coeff, 2)}, '
                f'p={round(pval, 5)})',
                weight=w,)
        axes[i_ft, i_mov].set_ylim(-3, 3)

        axes[i_ft, i_mov].set_xticks([1,2,3,4])
        axes[i_ft, i_mov].set_xticklabels(CDRS_categs.values())
        axes[i_ft, i_mov].set_ylabel('indiv. z-scored feature\n(a.u.)')
        axes[i_ft, i_mov].set_xlabel('LID (CDRS sum)')

plt.tight_layout()

plt.savefig(os.path.join(utilsFiles.get_project_path('figures'),
                            'feat_dysk_corrs',
                            'corr_boxplots',
                            FIG_NAME),
            dpi=300, facecolor='w',)

plt.close()
        

In [ ]:
# plot HEATMAP

FIG_NAME = '0129cf_heatmap_specPowerVar_vs_LIDcategs'


fig, ax = plt.subplots(1, 1,
                         figsize=(12, 4),)
ALPHA = .05 / len(feat_names['STN'])
# 0 is not sign, 1 is sign
heat_arrs = {sig_label: np.array([
    [np.nan] * len(feat_names['STN'])
] * 2) for sig_label in [0, 1]}

for i_ft, ft in enumerate(feat_names['STN']):
    
    for i_mov, MOV_SEL in enumerate(['INDEP', 'DEPEND']):
        # fill arrays with stats calculated before
        coeff = stat_arr[MOV_SEL]['coef'][i_ft]
        pval = stat_arr[MOV_SEL]['pval'][i_ft]
        sig_lab = (pval < ALPHA).astype(int)
        heat_arrs[sig_lab][i_mov, i_ft] = coeff

# non-sign heatmap
vmin, vmax = -.25, .25
cmap = 'coolwarm'  # RdBu_r
nonsig_map = ax.imshow(heat_arrs[0], vmin=vmin, vmax=vmax,
                           cmap=cmap, )
# hatch = plt.pcolor(heat_arrs[0], vmin=vmin, vmax=vmax,
#                    hatch='//', cmap=cmap,
#                    edgecolor='w', )

# ax.imshow(X=heat_arrs[0], cmap='coolwarm',
        #   alpha=.6, vmin=-.3, vmax=.3,)
sig_map = ax.imshow(X=heat_arrs[1], cmap=cmap,  # RdBu_r
                    alpha=.9, vmin=vmin, vmax=vmax,)

for i_m, i_f in product(np.arange(heat_arrs[1].shape[0]),
                        np.arange(heat_arrs[1].shape[1])):
    if np.isnan(heat_arrs[1][i_m, i_f]): continue
    if abs(heat_arrs[1][i_m, i_f]) > .3: c='w'
    else: c = 'black'
    ax.text(i_f, i_m, s=round(heat_arrs[1][i_m, i_f], 2),
            color=c, horizontalalignment='center',
            verticalalignment='center', weight='bold',)

cbar = fig.colorbar(sig_map, pad=.01)
cbar.ax.set_ylabel('LMM coefficient (a.u.)')

ax.set_xticks(np.arange(len(feat_names['STN'])))
ax.set_xticklabels(feat_names['STN'],
                   rotation=75,)
ax.set_yticks([0, 1])
ax.set_yticklabels(['move-INDEPENDENT', 'move-DEPENDENT'],)

plt.tight_layout()

plt.savefig(os.path.join(utilsFiles.get_project_path('figures'),
                            'feat_dysk_corrs',
                            'corr_boxplots',
                            FIG_NAME),
            dpi=300, facecolor='w',)

plt.close()
        

In [ ]:
heat_arrs[1].shape